In [22]:
import networkx as nx
from tqdm import tqdm
from ge.models import deepwalk, line, node2vec
import numpy as np

In [14]:
path = './datas/train.csv'
ad_id, product_id, advertiser_id = [], [], []
with open(path, 'r') as f:
    for line in tqdm(f):
        line = line.strip()
        if not line:
            continue
        seqences = line.split()
        ad_id.append(seqences[2].split('_'))
        product_id.append(seqences[3].split('_'))
        advertiser_id.append(seqences[5].split('_'))

899999it [00:32, 27982.39it/s]


In [15]:
ad_id[0]

['3', '4', '5', '7', '13', '2', '8', '10', '9', '6', '11', '11', '12']

In [16]:
def save_edges(datas, path):
    with open(path, 'w') as f:
        for data in datas:
            if len(data) < 2:
                continue
            for i in range(len(data)-1):
                line = data[i] + ' ' + data[i+1] + '\n'
                f.write(line)

In [17]:
save_edges(ad_id, './datas/ad_id_edge_list.txt')
save_edges(product_id, './datas/product_id_edge_list.txt')
save_edges(advertiser_id, './datas/advertiser_id_edge_list.txt')

In [19]:
G = nx.read_edgelist('./datas/ad_id_edge_list.txt',
                     create_using=nx.DiGraph(), nodetype=None, data=[('weight', int)])

In [20]:
model = DeepWalk(G, walk_length=20, num_walks=1, workers=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 31.0min finished


In [21]:
model.train(embed_size=256, window_size=5, iter=3, workers=-1)

Learning embedding vectors...
Learning embedding vectors done!


In [23]:
def save_embeddings(embeddings, num_embeddings, embedding_dim, path):
    # 0 for pad, 1 for unk
    data = np.random.normal(loc=0.0, scale=1e-3, size=(num_embeddings, embedding_dim))
    data[0] = np.zeros(embedding_dim)
    for k, v in embeddings.items():
        data[int(k)] = v
    np.save(path, data)

In [24]:
save_embeddings(model.get_embeddings(), 579847, 256, './graph_embedding/ad_id_deepwalk')